In [31]:
import glob
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
# nltk.download('punkt')
# nltk.download('stopwords')

In [2]:
data_ig = pd.read_csv('datasets_instagram.csv')

In [3]:
data_ig.head()

,alt_imputed,firstComment_imputed,caption_imputed,times,owner,postingan,word_count,char_length
0,Photo by Dinas Pariwisata Lombok Utara in Dina...,🔥🔥🔥,[ FINALIS TERUNE DEDARE LOTIM 2023 ]\n\nHalo S...,2022-07-30 09:19:43+00:00,wonderfullomboksumbawa,[ FINALIS TERUNE DEDARE LOTIM 2023 ]\n\nHalo S...,82,758
1,Photo by Dinas Pariwisata Lombok Utara in Dina...,bang ini parkirnya dimana,Sahabat wajib coba sih. Karna dari bandara lom...,2024-06-29 23:28:09+00:00,lampaqlampaq,Sahabat wajib coba sih. Karna dari bandara lom...,95,551
2,Photo by Go Mandalika in Sukarare Lombok Tenga...,🔥🔥🔥,Ladies and Gentlemen\nTipak Sanak Saudara Sena...,2024-07-24 07:01:26+00:00,gomandalika,Ladies and Gentlemen\nTipak Sanak Saudara Sena...,154,1245
3,Photo by Dinas Pariwisata Lombok Utara in Dina...,🔥🔥🔥,"Car Free Night\n""PRAYA CITY POP""\n.\n.\n#cfn #...",2024-07-07 12:14:11+00:00,cathaproject.id,"Car Free Night\n""PRAYA CITY POP""\n.\n.\n#cfn #...",12,75
4,Photo by Dinas Pariwisata Lombok Utara in Dina...,💚,Pelatihan Pemandu Ekowisata 27-29 Juni 2024 te...,2024-06-29 10:00:39+00:00,gomandalika,Pelatihan Pemandu Ekowisata 27-29 Juni 2024 te...,16,143


In [5]:
print(data_ig.columns)


Index(['alt_imputed', 'firstComment_imputed', 'caption_imputed', 'times',
       'owner', 'postingan', 'word_count', 'char_length'],
      dtype='object')


In [6]:
data_ig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4692 entries, 0 to 4691
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   alt_imputed           4692 non-null   object
 1   firstComment_imputed  4692 non-null   object
 2   caption_imputed       4692 non-null   object
 3   times                 4692 non-null   object
 4   owner                 4692 non-null   object
 5   postingan             4692 non-null   object
 6   word_count            4692 non-null   int64 
 7   char_length           4692 non-null   int64 
dtypes: int64(2), object(6)
memory usage: 293.4+ KB


In [7]:
posting = data_ig['postingan'] + " " + data_ig['caption_imputed']
komentar = (
 data_ig['firstComment_imputed'].fillna('')
)
df_posting = pd.DataFrame(posting)
df_komentar = pd.DataFrame(komentar)

In [8]:
print(df_posting[0:10])

                                                   0
0  [ FINALIS TERUNE DEDARE LOTIM 2023 ]\n\nHalo S...
1  Sahabat wajib coba sih. Karna dari bandara lom...
2  Ladies and Gentlemen\nTipak Sanak Saudara Sena...
3  Car Free Night\n"PRAYA CITY POP"\n.\n.\n#cfn #...
4  Pelatihan Pemandu Ekowisata 27-29 Juni 2024 te...
5  Di Lombok banyak tempat wisata yang bisa kalia...
6  Hallo #BaturMingo \n\nAlam memang tidak pernah...
7  Bukan lagi prihal warisan, Kebudayaan mencermi...
8  hallo #baturmingo\nawal bulan waktunya kumpul ...
9  Jangan lupa datang ya guys 🕺💃 Jangan lupa data...


In [9]:
gabung = pd.concat([df_posting, df_komentar], axis=1)
gabung.columns = ['Posting', 'Komentar']

In [10]:
gabung.head()

,Posting,Komentar
0,[ FINALIS TERUNE DEDARE LOTIM 2023 ]\n\nHalo S...,🔥🔥🔥
1,Sahabat wajib coba sih. Karna dari bandara lom...,bang ini parkirnya dimana
2,Ladies and Gentlemen\nTipak Sanak Saudara Sena...,🔥🔥🔥
3,"Car Free Night\n""PRAYA CITY POP""\n.\n.\n#cfn #...",🔥🔥🔥
4,Pelatihan Pemandu Ekowisata 27-29 Juni 2024 te...,💚


In [11]:
null_cols = gabung.columns[gabung.isnull().any()] # Mencari kolom yang memiliki nilai null
null_df = gabung[null_cols].isnull().sum().to_frame(name='Null Count')\
 .merge(gabung[null_cols].isnull().mean().mul(100).to_frame(name='Null Percent'), left_index=True, right_index=True)
null_df_sorted = null_df.sort_values(by='Null Count', ascending=False) # Mengurutkan berdasarkan 'Null Count'
print(null_df_sorted)

Empty DataFrame
Columns: [Null Count, Null Percent]
Index: []


In [13]:
count_duplicates = gabung[gabung.duplicated()].shape[0]
print("Number of duplicate rows:", count_duplicates)

Number of duplicate rows: 269


In [14]:
gabung.drop_duplicates(inplace=True)

In [15]:
gabung.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4423 entries, 0 to 4691
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Posting   4423 non-null   object
 1   Komentar  4423 non-null   object
dtypes: object(2)
memory usage: 103.7+ KB


In [17]:
gabung['Posting'].isnull().sum()
gabung['Posting'].fillna('', inplace=True)
gabung['Posting'].isnull().sum()

0

In [18]:
gabung

,Posting,Komentar
0,[ FINALIS TERUNE DEDARE LOTIM 2023 ]\n\nHalo S...,🔥🔥🔥
1,Sahabat wajib coba sih. Karna dari bandara lom...,bang ini parkirnya dimana
2,Ladies and Gentlemen\nTipak Sanak Saudara Sena...,🔥🔥🔥
3,"Car Free Night\n""PRAYA CITY POP""\n.\n.\n#cfn #...",🔥🔥🔥
4,Pelatihan Pemandu Ekowisata 27-29 Juni 2024 te...,💚
...,...,...
4687,#Repost @putriwithlove\n・・・\nEnjoying a beauti...,🔥🔥🔥
4688,#Repost @pondoksantiestate\n・・・\nIt’s time for...,@carissazerlinda honeymoon lagi ? Pulang bawa ...
4689,#Repost @steven.alexandre\n・・・\nSchooling West...,🔥🔥🔥
4690,#Repost @hotellumi\n・・・\nLush green haven awai...,💚🌴🙌🏼😎


In [19]:
gabung['posting_lower_case'] = gabung['Posting'].apply(lambda x: x.lower() if isinstance(x, str) else x)
gabung.head()

,Posting,Komentar,posting_lower_case
0,[ FINALIS TERUNE DEDARE LOTIM 2023 ]\n\nHalo S...,🔥🔥🔥,[ finalis terune dedare lotim 2023 ]\n\nhalo s...
1,Sahabat wajib coba sih. Karna dari bandara lom...,bang ini parkirnya dimana,sahabat wajib coba sih. karna dari bandara lom...
2,Ladies and Gentlemen\nTipak Sanak Saudara Sena...,🔥🔥🔥,ladies and gentlemen\ntipak sanak saudara sena...
3,"Car Free Night\n""PRAYA CITY POP""\n.\n.\n#cfn #...",🔥🔥🔥,"car free night\n""praya city pop""\n.\n.\n#cfn #..."
4,Pelatihan Pemandu Ekowisata 27-29 Juni 2024 te...,💚,pelatihan pemandu ekowisata 27-29 juni 2024 te...


In [20]:
import re
from bs4 import BeautifulSoup
#fungsi untuk menghapus URL dan HTML
def remove_html_tags(text):
 #hapus url
 text = re.sub(r'http\S+|www.\S+', '', text)
 soup = BeautifulSoup(text, "html.parser")
 return soup.get_text()
gabung['remove_html'] = gabung['posting_lower_case'].apply(remove_html_tags)

C:\Users\Nora\AppData\Local\Temp\ipykernel_5084\947884154.py:7: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [21]:
gabung.head()

,Posting,Komentar,posting_lower_case,remove_html
0,[ FINALIS TERUNE DEDARE LOTIM 2023 ]\n\nHalo S...,🔥🔥🔥,[ finalis terune dedare lotim 2023 ]\n\nhalo s...,[ finalis terune dedare lotim 2023 ]\n\nhalo s...
1,Sahabat wajib coba sih. Karna dari bandara lom...,bang ini parkirnya dimana,sahabat wajib coba sih. karna dari bandara lom...,sahabat wajib coba sih. karna dari bandara lom...
2,Ladies and Gentlemen\nTipak Sanak Saudara Sena...,🔥🔥🔥,ladies and gentlemen\ntipak sanak saudara sena...,ladies and gentlemen\ntipak sanak saudara sena...
3,"Car Free Night\n""PRAYA CITY POP""\n.\n.\n#cfn #...",🔥🔥🔥,"car free night\n""praya city pop""\n.\n.\n#cfn #...","car free night\n""praya city pop""\n.\n.\n#cfn #..."
4,Pelatihan Pemandu Ekowisata 27-29 Juni 2024 te...,💚,pelatihan pemandu ekowisata 27-29 juni 2024 te...,pelatihan pemandu ekowisata 27-29 juni 2024 te...


In [22]:
import re
def remove_special_characters(text):
 text = re.sub(r'[^A-Za-z\s\n]', '', text)
 return text
gabung['remove_special_characters'] = gabung['remove_html'].apply(remove_special_characters)

In [23]:
gabung.head()

,Posting,Komentar,posting_lower_case,remove_html,remove_special_characters
0,[ FINALIS TERUNE DEDARE LOTIM 2023 ]\n\nHalo S...,🔥🔥🔥,[ finalis terune dedare lotim 2023 ]\n\nhalo s...,[ finalis terune dedare lotim 2023 ]\n\nhalo s...,finalis terune dedare lotim \n\nhalo sobat p...
1,Sahabat wajib coba sih. Karna dari bandara lom...,bang ini parkirnya dimana,sahabat wajib coba sih. karna dari bandara lom...,sahabat wajib coba sih. karna dari bandara lom...,sahabat wajib coba sih karna dari bandara lomb...
2,Ladies and Gentlemen\nTipak Sanak Saudara Sena...,🔥🔥🔥,ladies and gentlemen\ntipak sanak saudara sena...,ladies and gentlemen\ntipak sanak saudara sena...,ladies and gentlemen\ntipak sanak saudara sena...
3,"Car Free Night\n""PRAYA CITY POP""\n.\n.\n#cfn #...",🔥🔥🔥,"car free night\n""praya city pop""\n.\n.\n#cfn #...","car free night\n""praya city pop""\n.\n.\n#cfn #...",car free night\npraya city pop\n\n\ncfn carfre...
4,Pelatihan Pemandu Ekowisata 27-29 Juni 2024 te...,💚,pelatihan pemandu ekowisata 27-29 juni 2024 te...,pelatihan pemandu ekowisata 27-29 juni 2024 te...,pelatihan pemandu ekowisata juni telah seles...


In [24]:
def remove_escape_characters(text):
 return re.sub(r'[\n\r\t]', ' ', text)
gabung['remove_escape_characters'] = gabung['remove_special_characters'].apply(remove_escape_characters)

In [25]:
gabung.head()

,Posting,Komentar,posting_lower_case,remove_html,remove_special_characters,remove_escape_characters
0,[ FINALIS TERUNE DEDARE LOTIM 2023 ]\n\nHalo S...,🔥🔥🔥,[ finalis terune dedare lotim 2023 ]\n\nhalo s...,[ finalis terune dedare lotim 2023 ]\n\nhalo s...,finalis terune dedare lotim \n\nhalo sobat p...,finalis terune dedare lotim halo sobat pra...
1,Sahabat wajib coba sih. Karna dari bandara lom...,bang ini parkirnya dimana,sahabat wajib coba sih. karna dari bandara lom...,sahabat wajib coba sih. karna dari bandara lom...,sahabat wajib coba sih karna dari bandara lomb...,sahabat wajib coba sih karna dari bandara lomb...
2,Ladies and Gentlemen\nTipak Sanak Saudara Sena...,🔥🔥🔥,ladies and gentlemen\ntipak sanak saudara sena...,ladies and gentlemen\ntipak sanak saudara sena...,ladies and gentlemen\ntipak sanak saudara sena...,ladies and gentlemen tipak sanak saudara senam...
3,"Car Free Night\n""PRAYA CITY POP""\n.\n.\n#cfn #...",🔥🔥🔥,"car free night\n""praya city pop""\n.\n.\n#cfn #...","car free night\n""praya city pop""\n.\n.\n#cfn #...",car free night\npraya city pop\n\n\ncfn carfre...,car free night praya city pop cfn carfreenig...
4,Pelatihan Pemandu Ekowisata 27-29 Juni 2024 te...,💚,pelatihan pemandu ekowisata 27-29 juni 2024 te...,pelatihan pemandu ekowisata 27-29 juni 2024 te...,pelatihan pemandu ekowisata juni telah seles...,pelatihan pemandu ekowisata juni telah seles...


In [26]:
def normalize_whitespace(text):
 return ' '.join(text.split())
gabung['cleaned_space'] = gabung['remove_escape_characters']

In [27]:
gabung.head()

,Posting,Komentar,posting_lower_case,remove_html,remove_special_characters,remove_escape_characters,cleaned_space
0,[ FINALIS TERUNE DEDARE LOTIM 2023 ]\n\nHalo S...,🔥🔥🔥,[ finalis terune dedare lotim 2023 ]\n\nhalo s...,[ finalis terune dedare lotim 2023 ]\n\nhalo s...,finalis terune dedare lotim \n\nhalo sobat p...,finalis terune dedare lotim halo sobat pra...,finalis terune dedare lotim halo sobat pra...
1,Sahabat wajib coba sih. Karna dari bandara lom...,bang ini parkirnya dimana,sahabat wajib coba sih. karna dari bandara lom...,sahabat wajib coba sih. karna dari bandara lom...,sahabat wajib coba sih karna dari bandara lomb...,sahabat wajib coba sih karna dari bandara lomb...,sahabat wajib coba sih karna dari bandara lomb...
2,Ladies and Gentlemen\nTipak Sanak Saudara Sena...,🔥🔥🔥,ladies and gentlemen\ntipak sanak saudara sena...,ladies and gentlemen\ntipak sanak saudara sena...,ladies and gentlemen\ntipak sanak saudara sena...,ladies and gentlemen tipak sanak saudara senam...,ladies and gentlemen tipak sanak saudara senam...
3,"Car Free Night\n""PRAYA CITY POP""\n.\n.\n#cfn #...",🔥🔥🔥,"car free night\n""praya city pop""\n.\n.\n#cfn #...","car free night\n""praya city pop""\n.\n.\n#cfn #...",car free night\npraya city pop\n\n\ncfn carfre...,car free night praya city pop cfn carfreenig...,car free night praya city pop cfn carfreenig...
4,Pelatihan Pemandu Ekowisata 27-29 Juni 2024 te...,💚,pelatihan pemandu ekowisata 27-29 juni 2024 te...,pelatihan pemandu ekowisata 27-29 juni 2024 te...,pelatihan pemandu ekowisata juni telah seles...,pelatihan pemandu ekowisata juni telah seles...,pelatihan pemandu ekowisata juni telah seles...


In [50]:
# gabung['text_tokenized'] = gabung['cleaned_space'].apply(word_tokenize)
# gabung.head()
import nltk
from nltk.tokenize import word_tokenize

# Unduh tokenizer jika belum diunduh
nltk.download('punkt')

# Tokenisasi pada kolom 'cleaned_space'
gabung['text_tokenized'] = gabung['cleaned_space'].apply(word_tokenize)

# Melihat lima baris pertama dari hasil
gabung.head()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nora\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


,Posting,Komentar,posting_lower_case,remove_html,remove_special_characters,remove_escape_characters,cleaned_space,text_tokenized
0,[ FINALIS TERUNE DEDARE LOTIM 2023 ]\n\nHalo S...,🔥🔥🔥,[ finalis terune dedare lotim 2023 ]\n\nhalo s...,[ finalis terune dedare lotim 2023 ]\n\nhalo s...,finalis terune dedare lotim \n\nhalo sobat p...,finalis terune dedare lotim halo sobat pra...,finalis terune dedare lotim halo sobat pra...,"[finalis, terune, dedare, lotim, halo, sobat, ..."
1,Sahabat wajib coba sih. Karna dari bandara lom...,bang ini parkirnya dimana,sahabat wajib coba sih. karna dari bandara lom...,sahabat wajib coba sih. karna dari bandara lom...,sahabat wajib coba sih karna dari bandara lomb...,sahabat wajib coba sih karna dari bandara lomb...,sahabat wajib coba sih karna dari bandara lomb...,"[sahabat, wajib, coba, sih, karna, dari, banda..."
2,Ladies and Gentlemen\nTipak Sanak Saudara Sena...,🔥🔥🔥,ladies and gentlemen\ntipak sanak saudara sena...,ladies and gentlemen\ntipak sanak saudara sena...,ladies and gentlemen\ntipak sanak saudara sena...,ladies and gentlemen tipak sanak saudara senam...,ladies and gentlemen tipak sanak saudara senam...,"[ladies, and, gentlemen, tipak, sanak, saudara..."
3,"Car Free Night\n""PRAYA CITY POP""\n.\n.\n#cfn #...",🔥🔥🔥,"car free night\n""praya city pop""\n.\n.\n#cfn #...","car free night\n""praya city pop""\n.\n.\n#cfn #...",car free night\npraya city pop\n\n\ncfn carfre...,car free night praya city pop cfn carfreenig...,car free night praya city pop cfn carfreenig...,"[car, free, night, praya, city, pop, cfn, carf..."
4,Pelatihan Pemandu Ekowisata 27-29 Juni 2024 te...,💚,pelatihan pemandu ekowisata 27-29 juni 2024 te...,pelatihan pemandu ekowisata 27-29 juni 2024 te...,pelatihan pemandu ekowisata juni telah seles...,pelatihan pemandu ekowisata juni telah seles...,pelatihan pemandu ekowisata juni telah seles...,"[pelatihan, pemandu, ekowisata, juni, telah, s..."


In [51]:
nltk.download('stopwords')
stop_words_english = set(stopwords.words('english'))
stop_words_indonesia = set(stopwords.words('indonesian'))
def remove_stopwords_english(tokens):
 return [word for word in tokens if word.lower() not in stop_words_english]
def remove_stopwords_indonesia(tokens):
 return [word for word in tokens if word.lower() not in stop_words_indonesia]
gabung['remove_stopword_english'] = gabung['text_tokenized'].apply(remove_stopwords_english)
gabung['remove_stopword_indonesia'] = gabung['remove_stopword_english'].apply(remove_stopwords_indonesia)



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nora\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
gabung.head()

,Posting,Komentar,posting_lower_case,remove_html,remove_special_characters,remove_escape_characters,cleaned_space
0,[ FINALIS TERUNE DEDARE LOTIM 2023 ]\n\nHalo S...,🔥🔥🔥,[ finalis terune dedare lotim 2023 ]\n\nhalo s...,[ finalis terune dedare lotim 2023 ]\n\nhalo s...,finalis terune dedare lotim \n\nhalo sobat p...,finalis terune dedare lotim halo sobat pra...,finalis terune dedare lotim halo sobat pra...
1,Sahabat wajib coba sih. Karna dari bandara lom...,bang ini parkirnya dimana,sahabat wajib coba sih. karna dari bandara lom...,sahabat wajib coba sih. karna dari bandara lom...,sahabat wajib coba sih karna dari bandara lomb...,sahabat wajib coba sih karna dari bandara lomb...,sahabat wajib coba sih karna dari bandara lomb...
2,Ladies and Gentlemen\nTipak Sanak Saudara Sena...,🔥🔥🔥,ladies and gentlemen\ntipak sanak saudara sena...,ladies and gentlemen\ntipak sanak saudara sena...,ladies and gentlemen\ntipak sanak saudara sena...,ladies and gentlemen tipak sanak saudara senam...,ladies and gentlemen tipak sanak saudara senam...
3,"Car Free Night\n""PRAYA CITY POP""\n.\n.\n#cfn #...",🔥🔥🔥,"car free night\n""praya city pop""\n.\n.\n#cfn #...","car free night\n""praya city pop""\n.\n.\n#cfn #...",car free night\npraya city pop\n\n\ncfn carfre...,car free night praya city pop cfn carfreenig...,car free night praya city pop cfn carfreenig...
4,Pelatihan Pemandu Ekowisata 27-29 Juni 2024 te...,💚,pelatihan pemandu ekowisata 27-29 juni 2024 te...,pelatihan pemandu ekowisata 27-29 juni 2024 te...,pelatihan pemandu ekowisata juni telah seles...,pelatihan pemandu ekowisata juni telah seles...,pelatihan pemandu ekowisata juni telah seles...


In [35]:
# pip install Sastrawi
!pip install tqdm

In [56]:
class TalaStemmer:
    def __init__(self):
        self.prefixes = [
            'me', 'mem', 'men', 'meng', 'mend', 'meny',
            'ber', 'per', 'pe', 'di', 'se', 'ke', 'ter', 'di'
        ]
        self.suffixes = [
            'kan', 'i', 'an', 'lah', 'kah', 'ku', 'mu', 'nya', 's', 'es', 'in'
        ]
 
    def remove_prefix(self, word):
        for prefix in self.prefixes:
            if word.startswith(prefix):
                return word[len(prefix):]
        return word
 
    def remove_suffix(self, word):
        for suffix in self.suffixes:
            if word.endswith(suffix):
                return word[:-len(suffix)]
        return word
 
    def stem(self, words):
        # Menghilangkan prefiks dan sufiks untuk setiap kata dalam list
        return [self.remove_suffix(self.remove_prefix(word)) for word in words]

# Membuat objek stemmer
tala_stemmer = TalaStemmer()

# Mengaplikasikan stemming pada kolom 'remove_stopword_indonesia'
gabung['data_post_cleaned'] = gabung['remove_stopword_indonesia'].apply(tala_stemmer.stem)
gabung


,Posting,Komentar,posting_lower_case,remove_html,remove_special_characters,remove_escape_characters,cleaned_space,text_tokenized,remove_stopword_english,remove_stopword_indonesia,data_post_cleaned
0,[ FINALIS TERUNE DEDARE LOTIM 2023 ]\n\nHalo S...,🔥🔥🔥,[ finalis terune dedare lotim 2023 ]\n\nhalo s...,[ finalis terune dedare lotim 2023 ]\n\nhalo s...,finalis terune dedare lotim \n\nhalo sobat p...,finalis terune dedare lotim halo sobat pra...,finalis terune dedare lotim halo sobat pra...,"[finalis, terune, dedare, lotim, halo, sobat, ...","[finalis, terune, dedare, lotim, halo, sobat, ...","[finalis, terune, dedare, lotim, halo, sobat, ...","[finali, une, dedare, lotim, halo, sobat, prad..."
1,Sahabat wajib coba sih. Karna dari bandara lom...,bang ini parkirnya dimana,sahabat wajib coba sih. karna dari bandara lom...,sahabat wajib coba sih. karna dari bandara lom...,sahabat wajib coba sih karna dari bandara lomb...,sahabat wajib coba sih karna dari bandara lomb...,sahabat wajib coba sih karna dari bandara lomb...,"[sahabat, wajib, coba, sih, karna, dari, banda...","[sahabat, wajib, coba, sih, karna, dari, banda...","[sahabat, wajib, coba, sih, karna, bandara, lo...","[sahabat, wajib, coba, sih, karna, bandara, lo..."
2,Ladies and Gentlemen\nTipak Sanak Saudara Sena...,🔥🔥🔥,ladies and gentlemen\ntipak sanak saudara sena...,ladies and gentlemen\ntipak sanak saudara sena...,ladies and gentlemen\ntipak sanak saudara sena...,ladies and gentlemen tipak sanak saudara senam...,ladies and gentlemen tipak sanak saudara senam...,"[ladies, and, gentlemen, tipak, sanak, saudara...","[ladies, gentlemen, tipak, sanak, saudara, sen...","[ladies, gentlemen, tipak, sanak, saudara, sen...","[ladie, gentlemen, tipak, sanak, saudara, nami..."
3,"Car Free Night\n""PRAYA CITY POP""\n.\n.\n#cfn #...",🔥🔥🔥,"car free night\n""praya city pop""\n.\n.\n#cfn #...","car free night\n""praya city pop""\n.\n.\n#cfn #...",car free night\npraya city pop\n\n\ncfn carfre...,car free night praya city pop cfn carfreenig...,car free night praya city pop cfn carfreenig...,"[car, free, night, praya, city, pop, cfn, carf...","[car, free, night, praya, city, pop, cfn, carf...","[car, free, night, praya, city, pop, cfn, carf...","[car, free, night, praya, city, pop, cfn, carf..."
4,Pelatihan Pemandu Ekowisata 27-29 Juni 2024 te...,💚,pelatihan pemandu ekowisata 27-29 juni 2024 te...,pelatihan pemandu ekowisata 27-29 juni 2024 te...,pelatihan pemandu ekowisata juni telah seles...,pelatihan pemandu ekowisata juni telah seles...,pelatihan pemandu ekowisata juni telah seles...,"[pelatihan, pemandu, ekowisata, juni, telah, s...","[pelatihan, pemandu, ekowisata, juni, telah, s...","[pelatihan, pemandu, ekowisata, juni, selesai,...","[latih, mandu, ekowisata, jun, lesa, meton, mo..."
...,...,...,...,...,...,...,...,...,...,...,...
4687,#Repost @putriwithlove\n・・・\nEnjoying a beauti...,🔥🔥🔥,#repost @putriwithlove\n・・・\nenjoying a beauti...,#repost @putriwithlove\n・・・\nenjoying a beauti...,repost putriwithlove\n\nenjoying a beautiful v...,repost putriwithlove enjoying a beautiful vie...,repost putriwithlove enjoying a beautiful vie...,"[repost, putriwithlove, enjoying, a, beautiful...","[repost, putriwithlove, enjoying, beautiful, v...","[repost, putriwithlove, enjoying, beautiful, v...","[repost, putriwithlove, enjoying, beautiful, v..."
4688,#Repost @pondoksantiestate\n・・・\nIt’s time for...,@carissazerlinda honeymoon lagi ? Pulang bawa ...,#repost @pondoksantiestate\n・・・\nit’s time for...,#repost @pondoksantiestate\n・・・\nit’s time for...,repost pondoksantiestate\n\nits time for a par...,repost pondoksantiestate its time for a parad...,repost pondoksantiestate its time for a parad...,"[repost, pondoksantiestate, its, time, for, a,...","[repost, pondoksantiestate, time, paradise, gi...","[repost, pondoksantiestate, time, paradise, gi...","[repost, pondoksantiestate, time, paradise, gi..."
4689,#Repost @steven.alexandre\n・・・\nSchooling West...,🔥🔥🔥,#repost @steven.alexandre\n・・・\nschool

In [38]:
dataset_posting_cleaned = gabung
dataset_posting_cleaned.to_csv('dataset_posting_cleaned.csv', index=False)
print("Datasets hasil preprocessing berhasil disimpan")

Datasets hasil preprocessing berhasil disimpan


In [39]:
data_posting = pd.read_csv('dataset_posting_cleaned.csv')

In [ ]:
import pandas as pd
import re
# Fungsi untuk menghapus karakter tertentu (m, y, n, e, 'petik satu')
def hapus_karakter(teks):
 if isinstance(teks, list):
 teks = ' '.join(teks) # Gabungkan list menjadi string
 elif pd.isna(teks): # Cek jika teks adalah NaN
 return "" # Kembalikan string kosong jika NaN
 
 # Ganti karakter yang ingin dihapus termasuk tanda petik satu
 teks = teks.replace(' m ', ' ').replace(' y ', ' ').replace(' n ', ' ').replace(' e ', ' ').replace('[]', ' ').replace(',', ' ').replace(']', ' ').replace('[', ' ') 
 # Hapus tanda petik satu
 teks = teks.replace("'", "") # Menghapus semua tanda petik satu
 
 teks = re.sub(r'\s+', ' ', teks) # Hapus spasi berlebih
 return teks.strip() # Kembalikan teks yang telah dibersihkan
# Terapkan ke kolom 'data_post_cleaned'
data_posting['more_cleaned_konten'] = data_posting['data_post_cleaned'].apply(hapus_karakter)
data_posting['fix_kontent'] = data_posting['more_cleaned_konten'].apply(hapus_karakter)


In [42]:
data_posting

,Posting,Komentar,posting_lower_case,remove_html,remove_special_characters,remove_escape_characters,cleaned_space
0,[ FINALIS TERUNE DEDARE LOTIM 2023 ]\n\nHalo S...,🔥🔥🔥,[ finalis terune dedare lotim 2023 ]\n\nhalo s...,[ finalis terune dedare lotim 2023 ]\n\nhalo s...,finalis terune dedare lotim \n\nhalo sobat p...,finalis terune dedare lotim halo sobat pra...,finalis terune dedare lotim halo sobat pra...
1,Sahabat wajib coba sih. Karna dari bandara lom...,bang ini parkirnya dimana,sahabat wajib coba sih. karna dari bandara lom...,sahabat wajib coba sih. karna dari bandara lom...,sahabat wajib coba sih karna dari bandara lomb...,sahabat wajib coba sih karna dari bandara lomb...,sahabat wajib coba sih karna dari bandara lomb...
2,Ladies and Gentlemen\nTipak Sanak Saudara Sena...,🔥🔥🔥,ladies and gentlemen\ntipak sanak saudara sena...,ladies and gentlemen\ntipak sanak saudara sena...,ladies and gentlemen\ntipak sanak saudara sena...,ladies and gentlemen tipak sanak saudara senam...,ladies and gentlemen tipak sanak saudara senam...
3,"Car Free Night\n""PRAYA CITY POP""\n.\n.\n#cfn #...",🔥🔥🔥,"car free night\n""praya city pop""\n.\n.\n#cfn #...","car free night\n""praya city pop""\n.\n.\n#cfn #...",car free night\npraya city pop\n\n\ncfn carfre...,car free night praya city pop cfn carfreenig...,car free night praya city pop cfn carfreenig...
4,Pelatihan Pemandu Ekowisata 27-29 Juni 2024 te...,💚,pelatihan pemandu ekowisata 27-29 juni 2024 te...,pelatihan pemandu ekowisata 27-29 juni 2024 te...,pelatihan pemandu ekowisata juni telah seles...,pelatihan pemandu ekowisata juni telah seles...,pelatihan pemandu ekowisata juni telah seles...
...,...,...,...,...,...,...,...
4418,#Repost @putriwithlove\n・・・\nEnjoying a beauti...,🔥🔥🔥,#repost @putriwithlove\n・・・\nenjoying a beauti...,#repost @putriwithlove\n・・・\nenjoying a beauti...,repost putriwithlove\n\nenjoying a beautiful v...,repost putriwithlove enjoying a beautiful vie...,repost putriwithlove enjoying a beautiful vie...
4419,#Repost @pondoksantiestate\n・・・\nIt’s time for...,@carissazerlinda honeymoon lagi ? Pulang bawa ...,#repost @pondoksantiestate\n・・・\nit’s time for...,#repost @pondoksantiestate\n・・・\nit’s time for...,repost pondoksantiestate\n\nits time for a par...,repost pondoksantiestate its time for a parad...,repost pondoksantiestate its time for a parad...
4420,#Repost @steven.alexandre\n・・・\nSchooling West...,🔥🔥🔥,#repost @steven.alexandre\n・・・\nschooling west...,#repost @steven.alexandre\n・・・\nschooling west...,repost stevenalexandre\n\nschooling western bu...,repost stevenalexandre schooling western buff...,repost stevenalexandre schooling western buff...
4421,#Repost @hotellumi\n・・・\nLush green haven awai...,💚🌴🙌🏼😎,#repost @hotellumi\n・・・\nlush green haven awai...,#repost @hotellumi\n・・・\nlush green haven awai...,repost hotellumi\n\nlush green haven awaits o...,repost hotellumi lush green haven awaits our...,repost hotellumi lush green haven awaits our...


In [63]:
!pip install matplotlib wordcloud

In [70]:
# Import library yang diperlukan
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from collections import Counter
# Menggabungkan semua kata dalam variabel 'data_post_cleaned'
text_data = ' '.join(data_posting['fix_kontent'])
# Membuat objek WordCloud dan menghitung frekuensi kata
wordcloud = WordCloud(width=800, height=400, background_color='white', colormap='viridis').generate(text_data)
# Menghitung frekuensi kata secara manual menggunakan Counter
word_counts = Counter(text_data.split())
# Menampilkan WordCloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off') # Menghilangkan sumbu
plt.show()
# Menampilkan frekuensi kata
print("Word frequencies:")
for word, count in word_counts.most_common(20): # Menampilkan 10 kata paling sering muncul
 print(f"{word}: {count}")

TypeError: sequence item 3: expected str instance, float found

In [71]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Inisialisasi TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=20) # max_features membatasi kata-kata paling penting (misal 1000 kata)
# Menerapkan TF-IDF pada kolom 'data_post_cleaned'
tfidf_matrix = tfidf_vectorizer.fit_transform(data_posting['fix_kontent'])
# Mengubah hasil TF-IDF menjadi DataFrame agar lebih mudah dianalisis
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
# Menampilkan 5 baris pertama dari hasil TF-IDF
print(tfidf_df)


ValueError: np.nan is an invalid document, expected byte or unicode string.

In [72]:
mean_tfidf = tfidf_df.mean().sort_values(ascending=False)
top_n = 20
top_words = mean_tfidf.head(top_n)
top_words

NameError: name 'tfidf_df' is not defined

In [55]:
colors = plt.cm.viridis(np.linspace(0, 1, top_n))
plt.figure(figsize=(10, 5))
plt.bar(top_words.index, top_words.values, color=colors)
plt.title(f'Top {top_n} Kata dengan nilai TF-IDF Tertinggi')
plt.xlabel('Kata')
plt.ylabel('Nilai TF-IDF Rata-rata')
plt.xticks(rotation=75)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

NameError: name 'top_n' is not defined